In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

print(tf.test.is_gpu_available())

True


In [3]:
from DataGenerator import MidiDataGenerator

midi = MidiDataGenerator('./raw', m=16)
x_train, pos = midi.samples(size=20000)
print(x_train.shape)

(20000, 16, 96, 96)


## Autoencorder

In [4]:
x_shape = x_train.shape

In [5]:
x = keras.layers.Input(shape=x_shape[1:])

y = keras.layers.Reshape(target_shape=(x_shape[1], x_shape[2]*x_shape[3]))(x)
y = keras.layers.TimeDistributed(keras.layers.Dense(2000, activation='relu'))(y)
y = keras.layers.TimeDistributed(keras.layers.Dense(200, activation='relu'))(y)
y = keras.layers.Flatten()(y)
y = keras.layers.Dense(1600, activation='relu')(y)

y = keras.layers.Dense(120)(y)
y = keras.layers.BatchNormalization(momentum=0.9)(y)

y = keras.layers.Dense(1600, name='encoder')(y)
y = keras.layers.BatchNormalization(momentum=0.9)(y)
y = keras.layers.Activation('relu')(y)

y = keras.layers.Dense(16*200)(y)
y = keras.layers.Reshape(target_shape=(16, 200))(y)
y = keras.layers.TimeDistributed(keras.layers.BatchNormalization(momentum=0.9))(y)
y = keras.layers.Activation('relu')(y)

y = keras.layers.TimeDistributed(keras.layers.Dense(2000))(y)
y = keras.layers.TimeDistributed(keras.layers.BatchNormalization(momentum=0.9))(y)
y = keras.layers.Activation('relu')(y)

y = keras.layers.TimeDistributed(keras.layers.Dense(x_shape[2]*x_shape[3], activation='sigmoid'))(y)
y = keras.layers.Reshape(target_shape=(x_shape[1], x_shape[2], x_shape[3]))(y)

print(x)
print(y)

Tensor("input_1:0", shape=(?, 16, 96, 96), dtype=float32)
Tensor("reshape_2/Reshape:0", shape=(?, 16, 96, 96), dtype=float32)


In [6]:
autoencorder = keras.Model(x, y)
autoencorder.compile(
    optimizer=keras.optimizers.RMSprop(lr=1e-3), 
    loss='binary_crossentropy',
)

In [7]:
his = autoencorder.fit(x_train, x_train, batch_size=200, epochs=1)

Epoch 1/1
20000/20000 [==============================] - 40s 2ms/step - loss: 5.5132


In [ ]:
x_test = midi.samples(size=200)

In [ ]:
y_test = autoencorder.predict(x_test)

In [ ]:
np.round(y_test, 4)

In [10]:
his.history['loss']

[5.513155741691589]